In [3]:
# Remember you need to use SWAN with NXCALS support....
from datetime import datetime
from datetime import timedelta
import time
#import lmfit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as msql
#old import datetime

import pytimber
ldb = pytimber.LoggingDB(source="nxcals")
#ldb = pytimber.LoggingDB(spark_session = spark)
#ldb = pytimber.LoggingDB(spark_session = spark, sparkconf="large",
#    sparkprops={
#        "spark.driver.cores", "4",
#        "spark.executor.memory", "10g",
#        "spark.sql.parquet.columnarReaderBatchSize", "32",
#        "spark.driver.maxResultSize", "1t",
#        "spark.task.maxDirectResultSize", "1t"
#})

In [2]:
# Just some simple imports   #import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams['lines.linewidth'] = 2

# search variables in the DB   #ldb.search("%totalIntensitySingle%")

In [ ]:
# One night data where SEM in Injection line were in:  # t1 = '2023-10-09 11:25:00.000'  # t2 = '2023-10-09 11:35:00.000'
# List of pickup signals
BeamInjection = 'LNR.APULB.0227:Acquisition:totalIntensitySingle' # Inj into AD
BeamExtrALPH  = 'LNE.APULB.0030:Acquisition:totalIntensitySingle' # Extr into 00 beam line (alpha,aegis,base ...
BeamExtrGBAR  = 'LNE.APULB.5030:Acquisition:totalIntensitySingle' # Extr into 50 beam line (gbar, puma ...

gbdb = msql.connect(host='pcnometh28.dyndns.cern.ch',database='gbar',user='gbar',password='To2023moko') 
if gbdb.is_connected():
    print('Connected to Gbar mysql database')
    cursor = gbdb.cursor()     #cursor = gbsc.cursor(buffered=True)

d    = timedelta(minutes=3)
ltrg = 0.0
cursor.execute("UPDATE man SET goElena =1")
gbdb.commit()
go   = True

while go: 
#for i in range(10):  # 240
    now  = datetime.now()  
    dat  = ldb.get([BeamExtrGBAR,BeamExtrALPH,BeamInjection],now-d,now+d)
    kys  = list(dat)
    ntrg = dat[kys[1]][0][-1]
    cursor.execute("SELECT goElena FROM man")
    go   = bool(cursor.fetchone()[0])  # bool(1) = True bool(0) = False  int(bool(1)) = 1
    gbdb.commit()
    #print(go)
    if ntrg > ltrg and go:
        dt = datetime.fromtimestamp(round(ntrg,0))
        #print("we have a new trigger",dt,dat[kys[1]][0][-1],dat[kys[1]][1][-1])  # second key, timestmp, last element I
        ltrg = ntrg
        
        # dict_keys(['LNE.APULB.0030:Acquisition:totalIntensitySingle',
        #            'LNE.APULB.5030:Acquisition:totalIntensitySingle',
        #            'LNR.APULB.0227:Acquisition:totalIntensitySingle'])    
        
        cursor.execute("UPDATE elena SET extr03tme =" + str(dat[kys[0]][0][-1]))
        cursor.execute("UPDATE elena SET extr03sig =" + str(dat[kys[0]][1][-1]))  
        cursor.execute("UPDATE elena SET extr53tme =" + str(dat[kys[1]][0][-1]))
        cursor.execute("UPDATE elena SET extr53sig =" + str(dat[kys[1]][1][-1]))    
        cursor.execute("UPDATE elena SET inj227tme =" + str(dat[kys[2]][0][-1]))    
        cursor.execute("UPDATE elena SET inj227sig =" + str(dat[kys[2]][1][-1]))    
        gbdb.commit()
        
    time.sleep(2)            
    
if gbdb is not None and gbsc.is_connected():
    gbdb.close()
    print('Connection mysql closed.')    
    
print("finished")


Connected to Gbar mysql database


In [3]:
#import pandas
import mysql.connector as msql
import datetime


# mysql -u gbar -pTo2023moko gbar -Bse "SELECT newdata FROM man;"

gbdb = msql.connect(host='pcnometh28.dyndns.cern.ch',database='gbar',user='gbar',password='To2023moko') 
if gbdb.is_connected():
    print('Connected to Gbar SC MySQL database')
    #cursor = gbsc.cursor(buffered=True)
    cursor = gbdb.cursor()
    #$cursor.execute("SELECT cpdate FROM man")
    #print(cursor.fetchall())
    upd_cpd = ("UPDATE elena SET extr53tme =" + str(1696864386.533))    
    cursor.execute(upd_cpd)
    gbdb.commit()
     
    #WHERE `run` = '345' AND `go` = '0' AND `stop` = '1' AND `nextTr` = '0' AND `newdata` = '1' AND `cpdate` = '1676042673' AND `Pb_on` = '12345678' AND `Hm_on` = '987654321' LIMIT 1;
  
if gbdb is not None and gbsc.is_connected():
    gbdb.close()
    print('Connection closed.')
    
# extr03tme extr03sig extr53tme extr53sig inj227tme inj227sig 
  
    #WHERE `run` = '345' AND `go` = '0' AND `stop` = '1' AND `nextTr` = '0' 
    #AND `newdata` = '1' AND `cpdate` = '1676042673' AND `Pb_on` = '12345678' AND `Hm_on` = '987654321' LIMIT 1;
     
    #print(dat['LNE.APULB.5030:Acquisition:totalIntensitySingle'][0][-1])

    #adat = dat.values()
    #print(adat)

    #for v in dat.values():
        #print(len(v))
        #print(v[0][-1],v[1][-1])
    
#dict_keys(['LNE.APULB.0030:Acquisition:totalIntensitySingle'])
#dict_items([('LNE.APULB.0030:Acquisition:totalIntensitySingle', (array([1.69684355e+09, 1.69684366e+09, 1.69684378e+09, 1.69684389e+09,
#       1.69684401e+09]), array([7.89372444, 8.47775269, 8.46801567, 8.59337997, 8.61419678])))])
#dict_values([(array([1.69684355e+09, 1.69684366e+09, 1.69684378e+09, 1.69684389e+09,
#       1.69684401e+09]), array([7.89372444, 8.47775269, 8.46801567, 8.59337997, 8.61419678]))])

#print("now =", now )  now = 2023-10-09 13:42:41.125025  
#dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
#print("date and time =", dt_string)  # date and time = 09/10/2023 13:42:41

    #print("selected key: ",kys[1])
    #print(type(dat))
    #print(dat.keys())
    #print(dat.items())
    #for k in dat:
    #    print(k)



#cursor = gbsc.cursor() cursor.execute("SELECT * FROM temp_C") data = np.asarray(cursor.fetchall()) evs = len(data)
#print("Total rows are: ", evs)
#print(data[evs-1,0])
#date = datetime.datetime.fromtimestamp(data[evs-1,0])
#print(date.strftime('Last entry: %Y-%m-%d %H:%M:%S'))
#cursor.close()

    
#import matplotlib.pyplot as plt    
#fig, ax = plt.subplots()
#ax.plot(data[:,0],data[:,1],'.')
#plt.show()

Connected to Gbar SC MySQL database
Connection closed.
1696864386.533


In [119]:

# old

#print(type(dat.values()))    
    
#print(dat['LNE.APULB.0030:Acquisition:totalIntensitySingle'])

#with open("simple.txt", "w") as fil:
#    fil.write("Hello world \n")
        
#fil = open("simple.txt", "w")
#for x in dat.items():
#  print(x)
#  fil.write(str(x))
#fil.close()    
    
#df = pd.DataFrame(dat.values())
#df.to_csv('test.txt',sep=',')
#df.to_csv('test.txt',sep='\t')


#BeamInjection = 'LNE.APULB.5030:INTENSITY'
#ADExtraction = 'DR.RF.NB-PART.100M_E:INTENSITY'

#SEMList_intensity = ['LNE50.BSGW.5020:FitAcq:gaussAmplitude','LNE50.BSGW.5060:FitAcq:gaussAmplitude']
#SEMList_position = ['LNE50.BSGW.5020:FitAcq:gaussMean','LNE50.BSGW.5060:FitAcq:gaussMean']
#SEMList_sigmas = ['LNE50.BSGW.5020:FitAcq:gaussSigma','LNE50.BSGW.5060:FitAcq:gaussSigma']

# GET the data from NXCALS via PyTimber
#SEM_intensity = ldb.get(SEMList_intensity, t1, t2)
#SEM_position  = ldb.get(SEMList_position, t1, t2)
#SEM_sigmas    = ldb.get(SEMList_sigmas, t1, t2)
#EXT_intensity = ldb.get(BeamInjection, t1,t2)
#AD_intensity = ldb.get(ADExtraction,t1,t2)

#InjInt = ldb.get(BeamInjection, t1,t2)
#AlpInt = ldb.get(BeamExtrALPH, t1,t2)
#GbrInt = ldb.get(BeamExtrGBAR, t1,t2)


Connected to Gbar SC MySQL database
[(1676042673,)]
Connection closed.


In [29]:
print(type(InjInt))
plt.plot(InjInt['LNE.APULB.0227:INTENSITY'])
#plt.plot(AD_intensity['DR.RF.NB-PART.100M_E:INTENSITY'][1])
plt.xlabel('shot number')
plt.ylabel('charge e+6')

# Intensity
plt.figure()
plt.plot(InjInt['LNR.APULB.0227:Acquisition:totalIntensitySingle'], label='LNR.APULB.0227')
#plt.plot(InjInt['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5060 H')
#plt.xlabel("shot number")
#plt.ylabel("Intensity [a.u.]")
plt.ylim([0, 2.5e3])
plt.legend()

NameError: name 'InjInt' is not defined

In [ ]:
# Intensity
plt.figure()
plt.plot(SEM_intensity['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_intensity['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("Intensity [a.u.]")
plt.ylim([0, 2.5e3])
plt.legend()

# Position
plt.figure()
plt.plot(SEM_position['LNE50.BSGW.5020:FitAcq:gaussMean'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_position['LNE50.BSGW.5020:FitAcq:gaussMean'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_position['LNE50.BSGW.5060:FitAcq:gaussMean'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_position['LNE50.BSGW.5060:FitAcq:gaussMean'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("<position> [mm]")
plt.ylim([-10, 10])
plt.legend()

# Sigma
plt.figure()
plt.plot(SEM_sigmas['LNE50.BSGW.5020:FitAcq:gaussSigma'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_sigmas['LNE50.BSGW.5020:FitAcq:gaussSigma'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_sigmas['LNE50.BSGW.5060:FitAcq:gaussSigma'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_sigmas['LNE50.BSGW.5060:FitAcq:gaussSigma'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("\sigma [mm]")
plt.ylim([0, 2.5])
plt.legend()


#Extraction Intensity
plt.figure()
plt.plot(EXT_intensity['LNE.APULB.5030:INTENSITY'][0], label='LNE.5030 V')
#plt.plot(EXT_intensity['LNE.APULB.5030:INTENSITY'][1], label='LNE.5030 H')
plt.xlabel('shot number')
plt.ylabel('Charges E6')
plt.legend()